# PII Inference Notebook

In [1]:
INFERENCE_MAX_LENGTH=1024
STRIDE=384
model_path = 'model_dir/DeBERTA_1024_Last'

In [2]:
import json
import argparse
from itertools import chain
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
import numpy as np

from src.utils import get_reference_df
from src.metric import compute_metrics
from src.data import tokenize_for_inference

## Tokenization

In [3]:
data = json.load(open("Dataset/valid.json"))
len(data)

10

In [4]:
ds = Dataset.from_dict(
    {
        "full_text": list(data["full_text"].values()),
        "document": list(data["document"].values()),
        "tokens": list(data["tokens"].values()),
        "trailing_whitespace": list(data["trailing_whitespace"].values()),
    }
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
ds = ds.map(
    tokenize_for_inference,
    fn_kwargs={
        "tokenizer": tokenizer,
        "max_length": INFERENCE_MAX_LENGTH,
        "stride": STRIDE,
    },
    num_proc=2,
)

Map (num_proc=2):   0%|          | 0/1350 [00:00<?, ? examples/s]

In [5]:
reference_df = get_reference_df(pd.DataFrame(data))

## Inferencing

In [6]:
model = AutoModelForTokenClassification.from_pretrained(model_path)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer = Trainer(
    model=model, 
    args=args, 
    data_collator=collator, 
    tokenizer=tokenizer,
)

## Ensemble

Originally from: https://www.kaggle.com/code/wonbulzhang/0-962-piidd-let-s-go-higher

In [7]:
import gc
import torch
import numpy as np
import time

from scipy.special import softmax

In [8]:
model_paths = {
    "model_dir/DeBERTA_1024_First": 1 / 3,
    "model_dir/DeBERTA_1024_Last": 1 / 3,
    "model_dir/DeBERTA_1024_Middle": 1 / 3,
}

In [9]:
def get_predictions(ds):
    all_preds = []

    for model_path, weight in model_paths.items():
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForTokenClassification.from_pretrained(model_path)
        collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
        args = TrainingArguments(
            ".",
            per_device_eval_batch_size=1,
            report_to="none",
        )
        trainer = Trainer(
            model=model,
            args=args,
            data_collator=collator,
            tokenizer=tokenizer,
        )
        predictions = trainer.predict(ds).predictions

        # This idea from this notebook: https://www.kaggle.com/code/olyatsimboy/912-blending-0-903-0-854-deberta3base
        weighted_predictions = softmax(predictions, axis=-1) * weight

        all_preds.append(weighted_predictions)
        del model, trainer
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(2)

    total_weight = sum(model_paths.values())
    weighted_average_predictions = np.sum(all_preds, axis=0) / total_weight
    return weighted_average_predictions

## Postprocessing

### Striding functions

As using the stride give an overlap in tokens, these have to be removed (either pick one side of the stride or average them, ...).

https://www.kaggle.com/code/valentinwerner/945-deberta-3-base-striding-inference/notebook

In [10]:
def backwards_map_preds(i, sub_predictions, max_len):
    # nothing to map backwards if sequence is too short to be split in the first place
    if max_len != 1:
        if i == 0:
            # First sequence needs no SEP token (used to end a sequence)
            sub_predictions = sub_predictions[:, :-1, :]
        elif i == max_len - 1:
            # End sequence needs to CLS token + Stride tokens
            sub_predictions = sub_predictions[
                :, 1 + STRIDE :, :
            ]  # CLS tokens + Stride tokens
        else:
            # Middle sequence needs to CLS token + Stride tokens + SEP token
            sub_predictions = sub_predictions[:, 1 + STRIDE : -1, :]

    return sub_predictions


def backwards_map_(i, row_attribute, max_len):
    # Same logics as for backwards_map_preds - except lists instead of 3darray
    if max_len != 1:
        if i == 0:
            row_attribute = row_attribute[:-1]
        elif i == max_len - 1:
            row_attribute = row_attribute[1 + STRIDE :]
        else:
            row_attribute = row_attribute[1 + STRIDE : -1]

    return row_attribute

In [11]:
preds = []
ds_dict = {"document": [], "token_map": [], "offset_mapping": [], "tokens": []}

for row in ds:
    # keys that need to be re-assembled
    row_preds = []
    row_offset = []

    for i, y in enumerate(row["offset_mapping"]):
        # create new datasset for each of of the splits per document
        x = Dataset.from_dict(
            {
                "token_type_ids": [row["token_type_ids"][i]],
                "input_ids": [row["input_ids"][i]],
                "attention_mask": [row["attention_mask"][i]],
                "offset_mapping": [row["offset_mapping"][i]],
            }
        )

        # predict for that split
        pred = get_predictions(x)
        # pred = trainer.predict(x).predictions

        # removing the stride and additional CLS & SEP that are created
        row_preds.append(backwards_map_preds(i, pred, len(row["offset_mapping"])))
        row_offset += backwards_map_(i, y, len(row["offset_mapping"]))

    # Finalize row
    ds_dict["document"].append(row["document"])
    ds_dict["tokens"].append(row["tokens"])
    ds_dict["token_map"].append(row["token_map"])
    ds_dict["offset_mapping"].append(row_offset)

    # Finalize prediction collection by concattenating
    p_concat = np.concatenate(row_preds, axis=1)
    preds.append(p_concat)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
config = json.load(open(Path(model_path) / "config.json"))
id2label = config["id2label"]

# preds_final = []
# for p in preds:
#     predictions = p.argmax(-1)
#     predictions_without_O = p[:,:,:12].argmax(-1)
#     O_predictions = p[:,:,12]

#     threshold = 0.95
#     preds_final.append(np.where(O_predictions < threshold, predictions_without_O , predictions))

In [ ]:
compute_metrics(
    (preds.predictions, None),
    id2label,
    ds,
    reference_df,
    threshold=0.95,
)

NameError: name 'preds' is not defined